In [1]:
from bs4 import BeautifulSoup
from fnmatch import fnmatch
from PIL import Image
from PIL import ImageFile
from pathlib import Path
from shutil import copyfile

import base64
import imagehash
import mysql.connector
import os
import re

In [2]:
# GENERAL CONFIGS
ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp_pm"


# == Files config ==
MAIN_FOLDER = r"D:\dnmarchives\novo\piratemarket"

In [3]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

cursor.execute("DROP TABLE IF EXISTS products_tmp_cm")


In [4]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp_pm ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 't'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [5]:
def get_content(content):
    clear_content = ""

    if content is not None:
        raw_c = content.text

        clear_content = re.sub(r'[\t\r\n]', ' ', raw_c)
        clear_content = re.sub(' +',' ', clear_content).lstrip()
        
    return clear_content

In [6]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}")

    for file in os.listdir(folder):            
        if "index.php_s=items&id=" in file:
            try:
                page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())
            except:
                page_content = None

            print(f"{folder}\{file}")

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                
                desc_content = page_content.findAll("div", attrs={"class": "form-add"})

                if desc_content is not None and len(desc_content) > 0:
                    try:
                        desc_div = desc_content[1]
                    except:
                        desc_div = ""
                else:
                    desc_div = page_content.find("div", attrs={"class": "buy_item_description"})
                    
                if desc_div and desc_div is not None:
                    description = get_content(desc_div)

                ## GET TITLE
                title = ""
                raw_title = page_content.find("div", attrs={"class": "product-name"})

                if raw_title is None:
                    raw_title = page_content.find("div", attrs={"class": "buy_item_name"})
                    
                if raw_title is not None:
                    title = get_content(raw_title)

                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_pm (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
                    values = (title, description, "Pirate Market", "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    main_div = page_content.find("p", attrs={"class": "product-image"})
                    
                    if main_div is not None:
                        img_content = main_div.find("img")
                        
                        if img_content is not None:
                            img_src = img_content["src"]

                            if img_src is not None:
                                url_image = Path(f"{folder}\{img_src}")

                                try:
                                    # copy images to new directory, renaming with product id
                                    if url_image.exists():
                                        print(f"Image {product_id} is normal")

                                        dst_dir = Path(f"D:\images\_pm\{product_id}.jpeg")
                                        copyfile(url_image, dst_dir)
                                    else:
                                        print(f"Image {product_id} does not exists")
                                except:
                                    print(f"Image {product_id} has error. Except")
                    else:
                        img_content = main_div = page_content.find("img")
                        
                        if img_content is not None:
                            img = img_content["src"]
                            
                            if img is not None:
                                try:
                                    data = "" + img.replace("data:image/jpg;base64,", "")
                                    data = "" + data.replace("data:image/jpeg;base64,", "")
                                    data = "" + data.replace("data:image/png;base64,", "")
                                    imgdata = base64.b64decode(data)

                                    img_filename = Path(f"D:\images\_pm\{product_id}.jpeg")

                                    with open(img_filename, 'wb') as f:
                                        f.write(imgdata)
                                        f.close
                                except:
                                    print(f"Image {product_id} has error. Except")

D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897935
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897936
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897937
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897938
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897939
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897940
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897941
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897942
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897943
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897944
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897945
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897947
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7897948
D:\dnmarchives\novo\piratemarket\2013-12-26\index.p

D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898090
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898091
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898092
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898093
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898094
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898095
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898096
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898097
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898098
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898099
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898100
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898101
D:\dnmarchives\novo\piratemarket\2013-12-26\index.php_s=items&id=7898102
D:\dnmarchives\novo\piratemarket\2013-12-26\index.p

D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898096
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898097
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898098
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898099
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898100
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898101
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898102
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898103
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898106
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898107
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898109
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898110
D:\dnmarchives\novo\piratemarket\2014-01-01\index.php_s=items&id=7898112
D:\dnmarchives\novo\piratemarket\2014-01-01\index.p

D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898076
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898077
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898078
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898079
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898080
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898081
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898082
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898083
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898083&page=1
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898085
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898086
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898087
D:\dnmarchives\novo\piratemarket\2014-01-09\index.php_s=items&id=7898088
D:\dnmarchives\novo\piratemarket\2014-01-09\

D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7897983&page=1
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7897984
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7897985
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7897986
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7897987
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7897988
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7897993
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7897994
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7897995
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7897998
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7897999
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7898000
D:\dnmarchives\novo\piratemarket\2014-01-15\index.php_s=items&id=7898003
D:\dnmarchives\novo\piratemarket\2014-01-15\

...

D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898872
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898873
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898874
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898875
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898877
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898878
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898879
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898880
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898881
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898884
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898885
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898886
D:\dnmarchives\novo\piratemarket\2014-09-20\index.php_s=items&id=7898887
D:\dnmarchives\novo\piratemarket\2014-09-20\index.p

D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898094&page=1
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898095
Image 11214 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898095&page=1
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898096
Image 11215 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898096&page=1
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898097
Image 11216 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898097&page=1
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898098
Image 11217 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898099
Image 11218 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898100
Image 11219 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&

D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898586&page=1
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898614
Image 11266 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898614&page=1
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898628
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898629
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898646
Image 11267 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898646&page=1
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898647
Image 11268 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898652
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898658
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7898663
Image 11269 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=

D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899010
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899029
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899044
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899143
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899153
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899163
Image 11322 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899163&page=1
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899165
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899167
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899168
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899169
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899170
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899171
D:\dnmarchives

D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899731
Image 11359 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899732
Image 11360 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899733
Image 11361 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899734
Image 11362 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899735
Image 11363 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899736
Image 11364 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899737
Image 11365 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899738
Image 11366 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899739
Image 11367 has error. Except
D:\dnmarchives\novo\piratemarket\2014-09-21\index.php_s=items&id=7899740
